# Understanding Simulated Annealing and the Quasar Solver

This notebook provides an initial walkthrough of the Simulated Annealing (SA) process. We'll use the `quasar-solver` to solve a small, artificial QUBO problem and visualize the results to build an intuition for how the algorithm works.

## What is Simulated Annealing?

Simulated Annealing (SA) is a heuristic algorithm used for finding a global optimum in a complex search space. It's a **probabilistic metaheuristic**, meaning it uses randomness to guide its search and doesn't guarantee the absolute best solution every time, but is excellent at finding very good ones efficiently.

The algorithm is inspired by the physical process of **annealing in metallurgy**, where a metal is heated to a high temperature and then slowly cooled. This process settles the metal's atoms into a very low-energy, highly-ordered crystalline structure.

SA uses this analogy to explore a problem's **energy landscape**—a map of all possible solutions and their corresponding costs (or "energy").

-   The algorithm starts at a hot initial temperature, allowing it to accept new solution candidates frequently in the beginning, even accepting "worse" solutions (uphill moves) to avoid getting trapped in **local minima**. This is the **exploration** phase.
-   As the algorithm progresses, the temperature gets reduced based on a controled cooling scheme. The lower the temperature, the lower the willingness to accept worse solutions. It begins to settle into the deepest valleys it has discovered. This is the **exploitation** phase.

### Key Parameters

The behavior of the solver is controlled by a few critical parameters:

* **Initial Temperature (`initial_temp`):** This is the starting temperature. It must be high enough to allow the solver to freely explore the entire energy landscape. If it's too low, the solver might get stuck in the first minimum it finds.
* **Final Temperature (`final_temp`):** The temperature at which the algorithm stops. It should be close to zero, ensuring the solver has "frozen" into a stable, low-energy solution.
* **Cooling Schedule (`schedule` & `schedule_params`):** This function dictates how the temperature decreases over time. The cooling rate is crucial: if it's too fast, the system gets "quenched" in a poor state; if it's too slow, the solver takes too long. Our solver uses a common **geometric schedule** where `T_new = alpha * T_old`, and `alpha` is the cooling rate.
* **Iterations Per Temperature (`iterations_per_temp`):** Also known as the **Markov chain length**. This is the number of steps the solver takes at each temperature level. It must be long enough for the system to reach a stable state ("thermal equilibrium") before cooling down further.

In [1]:
# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our solver components
from quasar_solver import QUBO, Solver

# Set a style for our plots
sns.set_theme(style="whitegrid")

## 1. The Optimization Problem

For the sake of this tutorial, we'll use a simple, yet non-trivial 4-variable QUBO with a known local minimum and a known global optimum. The goal is to find the binary vector $x$ that minimizes the energy $E = x^T Q x$.

We'll use the following $Q$ matrix:
$$
Q = \begin{pmatrix}
-2 & 3 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -2 & 3 \\
0 & 0 & 0 & -1 
\end{pmatrix}
$$

This matrix has two interesting configurations:
-   **Global Minimum:** The state **`[1, 0, 1, 0]`** has an energy of $Q_{00} + Q_{22} = -2 + (-2) = -4.0$. Any single bit flip from this state will result in a higher energy.
-   **Local Minimum:** The state **`[0, 1, 0, 1]`** has an energy of $Q_{11} + Q_{33} = -1 + (-1) = -2.0$. While not as good as -4.0, any single bit flip from *this* state also results in a higher energy, making it a stable "valley" in the energy landscape.

The off-diagonal values ($Q_{01}=3$ and $Q_{23}=3$) act as penalties. They make it energetically unfavorable to have both $x_0, x_1$ active at the same time, or both $x_2, x_3$ active. The challenge for the solver is to find its way out of the `-2.0` valley to discover the deeper `-4.0` one.

In [2]:
# Cell 2: Defining the QUBO Problem in Code
q_matrix = np.array([
    [-2, 3, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -2, 3],
    [0, 0, 0, -1]
])

# Create an instance of our QUBO class
problem = QUBO(q_matrix)

print("QUBO problem created successfully.")

QUBO problem created successfully.


## 2. Solving the Problem

Before we run the solver, we need to configure its parameters. These values control the annealing process and are crucial for balancing solution quality with runtime.

* `qubo`: This is the QUBO problem instance we just created.
* `initial_temp`: The starting temperature. We set it to `10.0`, which is high enough to allow the solver to explore freely and escape the local minimum trap.
* `final_temp`: The stopping temperature. A value of `0.1` is low enough to ensure the solver has settled into a stable, low-energy state.
* `iterations_per_temp`: The number of steps at each temperature (Markov chain length). `500` gives the system enough time to reach equilibrium before we cool it down further.
* `schedule_params`: A dictionary for the cooling schedule. We're using a geometric schedule, so we provide `{'alpha': 0.97}`. This means the temperature will be multiplied by 0.97 after each set of iterations, resulting in a slow, steady cooling process.

In [3]:
# Cell 3: Configuring the Solver
# We'll start with a reasonably high temperature and cool down slowly, applying the concepts we just discussed.

solver = Solver(
    qubo=problem,
    initial_temp=10.0,
    final_temp=0.1,
    iterations_per_temp=500,
    schedule_params={'alpha': 0.97} # Geometric cooling rate
)

print("Solver configured. Ready for detailed analysis.")

Solver configured. Ready for detailed analysis.


Now, we can simply call the `solve()` method to run the annealing process and get the result.

In [4]:
# Cell 4: Run the solver and print the result
result = solver.solve()

print(f"\nOptimal state found: {result.state}")
print(f"Lowest energy: {result.energy}")

Annealing complete. Final energy: -3.0

Optimal state found: [0 1 1 0]
Lowest energy: -3.0
